In [90]:
# environment set up
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd

# working folder
# data_dir = "/Users/Srikar/Desktop/Velexi/spectra-ml/data/"
data_dir = os.environ['DATA_DIR']
os.chdir(data_dir)

In [99]:
stddata_path = os.path.join(data_dir,"Srikar-Standardized")
metadata = pd.read_csv(os.path.join(stddata_path,"spectra-metadata.csv"), sep="|")
metadata.head()

,spectrum_id,value_type,material,spectrometer_purity_code,measurement_type,raw_data_path
0,23077,reflectance,Rangeland C03-022 S12% G22%,ASDFRa,AREF,ChapterV_Vegetation/splib07a_Rangeland_C03-022...
1,22344,reflectance,Marsh SPAL92%...a CRMS322v78,ASDFRa,AREF,ChapterV_Vegetation/splib07a_Marsh_SPAL92%...a...
2,22136,reflectance,Tumbleweed ANP92-2C Dry,BECKa,AREF,ChapterV_Vegetation/splib07a_Tumbleweed_ANP92-...
3,24017,reflectance,Rangeland L02-069 S00% G99%,ASDFRa,AREF,ChapterV_Vegetation/splib07a_Rangeland_L02-069...
4,21060,reflectance,Douglas-Fir YNP-DF-1 forest,AVIRISb,RTGC,ChapterV_Vegetation/splib07a_Douglas-Fir_YNP-D...


In [100]:
metadata = metadata[metadata['value_type'] == "reflectance"]
metadata = metadata[~metadata['spectrometer_purity_code'].str.contains("NIC4")]
metadata = metadata[metadata['raw_data_path'].str.contains("ChapterM")] # add in ChapterS Soils and Mixtures later
metadata.shape

(887, 6)

In [101]:
record_nums = []
y = []
spectrum_names = []

act = 0
aln = 0
chl = 0

for i in range(metadata.shape[0]):
    data = metadata.iloc[i, :]
    if data[2].find("Actinolite") != -1: # if material name contains actinolite
        record_nums.append(data[0])
        y.append(int(0))
        spectrum_names.append("Actinolite")
        act += 1
    elif data[2].find("Alun") != -1:
        record_nums.append(data[0])
        y.append(int(1))
        spectrum_names.append("Alunite")
        aln += 1
    elif (data[2].find("Chlorit") != -1 or data[2].find("Chlor.") != -1 or data[2].find("Chlor+") != -1 or data[2].find("Chl.") != -1):
        record_nums.append(data[0])
        y.append(int(2))
        spectrum_names.append("Chlorite")
        chl += 1

y = np.reshape(y, (len(y), 1))
num_samples = len(record_nums)
print(num_samples)
print(len(y))
print(type(y))
print(act)
print(aln)
print(chl)

40
40
<class 'numpy.ndarray'>
16
13
11


In [102]:
spectrum_len = 500
spectra = np.zeros((num_samples,spectrum_len))
wavelengths = np.zeros((1,spectrum_len))

In [103]:
num_neg = 0
for i in range(num_samples):
    hasnegative = False
    data = pd.read_csv(os.path.join(stddata_path,"{}.csv".format(record_nums[i])))
    if i == 0:
        wavelengths[i,:] = data.iloc[:, 0].to_numpy()
    spectra[i,:] = data.iloc[:, 1].to_numpy()
    for j in range(spectrum_len):
        if spectra[i,j] < 0:
            hasnegative = True
            spectra[i,j] = 0
    if hasnegative:
        print(record_nums[i])
        num_neg += 1
print(num_neg)

0


In [111]:
# --- plot the classes

# plot each class in a separate plot
# plot spectra names in legend
# plot minerals and mixtures w diff line widths

mineral_names = ["Actinolite", "Alunite", "Chlorite"]

# variables
num0 = 0 #number of samples of class 0
num1 = 0
num2 = 0

mineral_linewidth = 1         # linewidth = 1 is default
mixture_linewidth = 3         

# count the number of each class to make spectra0, spectra1, spectra2 databases
for i in range(num_samples):
    if y[i,0]== 0:
        num0 += 1
    elif y[i,0]== 1:
        num1 += 1
    elif y[i,0]== 2:
        num2 += 1

# make class-specific databases spectra0, ...1, ...2
spectra0 = np.zeros((num0,spectrum_len)) 
spectra1 = np.zeros((num1,spectrum_len)) 
spectra2 = np.zeros((num2,spectrum_len)) 

labels0 = ["" for x in range(num0)]
labels1 = ["" for x in range(num1)]
labels2 = ["" for x in range(num2)]

linewidth0 = np.zeros(num0)
linewidth1 = np.zeros(num1)
linewidth2 = np.zeros(num2)


# make counters for each database to place spectra
i0 = 0
i1 = 0
i2 = 0

# set linewidth for the spectrum 
# populate class-specific databases spectra0, ...1, ...2
for i in range(num_samples):
    
    # set linewidth
    #testcode
    #print(spectrum_categories)
    #print(spectrum_categories[i])
    
#     if spectrum_categories[i] == is_a_mineral:
#         linewidth = mineral_linewidth
        
#         #testcode
#         #print('min')
#     else: 
#         linewidth = mixture_linewidth
    linewidth = 2
        
        #testcode
        #print('mix')
    
    # populate matrices for making each class plot
    if y[i,0]== 0:
        spectra0[i0,:] = spectra[i,:]
        labels0[i0] = spectrum_names[i]
        linewidth0[i0] = linewidth
        i0 +=1
    elif y[i,0]== 1:
        spectra1[i1,:] = spectra[i,:]
        labels1[i1] = spectrum_names[i]
        linewidth1[i1] = linewidth
        i1 +=1
    else:
        spectra2[i2,:] = spectra[i,:]
        labels2[i2] = spectrum_names[i]
        linewidth2[i2] = linewidth
        i2 +=1

# plot each class-specific database separately
# remove linewidth for all mixtures/minerals to be standard
for i in range(i0):
#     plt.plot(range(1, spectrum_len+1), spectra0[i,:], label = labels0[i], linewidth = linewidth0[i])
    fig = plt.figure()
    plt.plot(wavelengths[0,:], spectra0[i,:], label = labels0[i], linewidth = linewidth0[i], color='k')
    plt.xticks([])
    plt.yticks([])
#     fig.patch.set_visible(False)
#     plt.show()
    path = os.path.join(data_dir, "plots", mineral_names[0] + str(i+1) + ".png")
    ax = fig.axes
    ax[0].axis('off')
    fig.savefig(path, format = "PNG")
    plt.close(fig)
# plt.legend(bbox_to_anchor=(1.1, 1.05))
# plt.show()

for i in range(i1):
#     plt.plot(range(1, spectrum_len+1), spectra1[i,:], label = labels1[i], linewidth = linewidth1[i])
    fig = plt.figure()
    plt.plot(wavelengths[0,:], spectra1[i,:], label = labels1[i], linewidth = linewidth1[i], color='k')
    plt.xticks([])
    plt.yticks([])
#     fig.patch.set_visible(False)
#     plt.show()
    path = os.path.join(data_dir, "plots", mineral_names[1] + str(i+1) + ".png")
    ax = fig.axes
    ax[0].axis('off')
    fig.savefig(path, format = "PNG")
    plt.close(fig)
# plt.legend(bbox_to_anchor=(1.1, 1.05))
# plt.show()

for i in range(i2):
#     plt.plot(range(1, spectrum_len+1), spectra2[i,:], label = labels2[i], linewidth = linewidth2[i])
    fig = plt.figure()
    plt.plot(wavelengths[0,:], spectra2[i,:], label = labels2[i], linewidth = linewidth2[i], color='k')
    plt.xticks([])
    plt.yticks([])
#     fig.patch.set_visible(False)
#     plt.show()
    path = os.path.join(data_dir, "plots", mineral_names[2] + str(i+1) + ".png")
    ax = fig.axes
    ax[0].axis('off')
    fig.savefig(path, format = "PNG")
    plt.close(fig)
# plt.legend(bbox_to_anchor=(1.1, 1.05))
# plt.show()

In [106]:
num_neg = 0
for file in os.listdir(stddata_path):
    data = pd.read_csv(os.path.join(stddata_path,file))
    if data.shape[1] == 2:
        arr = data.iloc[:, 1].to_numpy()
    if np.isnan(arr[0]) or np.isnan(arr[len(arr)-1]):
        print(file)
        num_neg += 1
        continue
#     for j in range(len(arr)):
#         if np.isnan(arr[j]):
#             print(file)
#             num_neg += 1
#             break
print(num_neg)

21242.csv
10921.csv
6796.csv
8750.csv
10706.csv
1984.csv
753.csv
21068.csv
5710.csv
4432.csv
2043.csv
960.csv
6543.csv
10510.csv
11785.csv
7932.csv
18945.csv
11034.csv
15246.csv
9062.csv
4963.csv
7517.csv
8142.csv
15078.csv
5329.csv
6620.csv
9513.csv
15723.csv
395.csv
1198.csv
22158.csv
10303.csv
8369.csv
1365.csv
18950.csv
9117.csv
785.csv
2718.csv
5936.csv
6581.csv
5088.csv
12676.csv
8962.csv
4816.csv
3148.csv
4592.csv
3216.csv
11751.csv
12073.csv
14402.csv
10664.csv
1170.csv
7267.csv
20994.csv
9706.csv
2121.csv
802.csv
2684.csv
625.csv
976.csv
21081.csv
18614.csv
6025.csv
7449.csv
1776.csv
8949.csv
6970.csv
2290.csv
9306.csv
10072.csv
5272.csv
769.csv
21046.csv
9662.csv
10264.csv
21085.csv
5851.csv
5689.csv
5892.csv
11768.csv
3950.csv
423.csv
5448.csv
9272.csv
5312.csv
8636.csv
13341.csv
3010.csv
344.csv
20985.csv
22162.csv
4541.csv
4353.csv
18639.csv
11184.csv
740.csv
7855.csv
8031.csv
12857.csv
3199.csv
1017.csv
8999.csv
1567.csv
2697.csv
18940.csv
6432.csv
7710.csv
1407.csv
1836.

In [107]:
metadata = metadata[metadata['value_type'] == "reflectance"]
metadata = metadata[~metadata['spectrometer_purity_code'].str.contains("NIC4")]
metadata = metadata[metadata['raw_data_path'].str.contains("ChapterM")]
metadata.shape

record_nums = []
mineral_names = []
for i in range(metadata.shape[0]):
    data = metadata.iloc[i, :]
    record_nums.append(data[0])
    mineral_names.append(data[2])

In [108]:
num_neg = 0
print(len(record_nums))
for i in range(len(record_nums)):
    data = pd.read_csv(os.path.join(stddata_path,"{}.csv".format(record_nums[i])))
    if data.shape[1] == 2:
        arr = data.iloc[:, 1].to_numpy()
    if np.isnan(arr[0]) or np.isnan(arr[len(arr)-1]):
        print(record_nums[i])
        print(mineral_names[i])
        num_neg += 1
        continue
#     for j in range(len(arr)):
#         if np.isnan(arr[j]):
#             print(file)
#             num_neg += 1
#             break
print(num_neg)

887
8927
Psilomelane HS139.1B
8962
Psilomelane HS139.4B
6792
Mascagnite GDS65.b (fr)
8943
Psilomelane HS139.3B
8933
Psilomelane HS139.2B
5


In [109]:
# os.listdir(stddata_path)

In [110]:
# data = pd.read_csv(os.path.join(stddata_path,"211.csv"))
# arr = data.iloc[:, 1].to_numpy()
# for j in range(spectrum_len):
#     print(type(arr[j]))